In [1]:
### user input (and autofill) for limited number of fruits/veggies/grains/nuts

### (A) user input for "where are you" (state) - future development, for now just go to centroid of the U.S.

### (B) user input for "where is your food from" (input country of origin, if U.S., then State, then city)

### (C) calculate distance between centroids (A and B)

### output: C

### (D) multiply distance by the amount of CO2 is generated by air, land, and 
### water travel (produce a range of possible emissions)

### output: D

### provide water use assuming that the net weight of given produce is 1 pound. Divide blue and total water use by 2000 
### because it's in metric tons and then multiply to generate how many gallons & liters are used

In [2]:
import pandas as pd
import numpy as np
import geopy as gp
from geopy import distance
from geopy.distance import great_circle

In [3]:
us_cities = pd.read_csv("C:/Users/Ashley/workspace/foodpact2/to_use/uscitiesv1.4.csv")

In [4]:
country_centroids = pd.read_csv("C:/Users/Ashley/workspace/foodpact2/to_use/country_centroids_az8.csv")

In [5]:
## cities_dict = pd.DataFrame(us_cities).stack().reset_index()
## cities_dict.columns = ['city', 'state_id', 'lat', 'lng']
## print(cities_dict)

#trying to sort out whether to convert dataframe to dictionary or not, really only need cit name, state, and associated
#lat and long to find great circle distance using geopy

In [6]:
## city and state format
your_location = input("Your location: ")
print(your_location)

Your location: Syracuse, NY
Syracuse, NY


In [8]:
us_cities['location'] = us_cities['city'] + ", " + us_cities['state_id']
us_cities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,population_proper,density,source,incorporated,timezone,zips,id,location
0,Prairie Ridge,Prairie Ridge,WA,Washington,53053,Pierce,47.1443,-122.1408,NaN,NaN,1349.8,polygon,False,America/Los_Angeles,98360 98391,1840037882,"Prairie Ridge, WA"
1,Edison,Edison,WA,Washington,53057,Skagit,48.5602,-122.4311,NaN,NaN,127.4,polygon,False,America/Los_Angeles,98232,1840017314,"Edison, WA"
2,Packwood,Packwood,WA,Washington,53041,Lewis,46.6085,-121.6702,NaN,NaN,213.9,polygon,False,America/Los_Angeles,98361,1840025265,"Packwood, WA"
3,Wautauga Beach,Wautauga Beach,WA,Washington,53035,Kitsap,47.5862,-122.5482,NaN,NaN,261.7,point,False,America/Los_Angeles,98366,1840037725,"Wautauga Beach, WA"
4,Harper,Harper,WA,Washington,53035,Kitsap,47.5207,-122.5196,NaN,NaN,342.1,point,False,America/Los_Angeles,98366,1840037659,"Harper, WA"


In [9]:
us_city_location = us_cities[['location', 'lat', 'lng']].copy()
us_city_location.head()

,location,lat,lng
0,"Prairie Ridge, WA",47.1443,-122.1408
1,"Edison, WA",48.5602,-122.4311
2,"Packwood, WA",46.6085,-121.6702
3,"Wautauga Beach, WA",47.5862,-122.5482
4,"Harper, WA",47.5207,-122.5196


In [10]:
us_city_location.rename(columns={'location': 'city'}, inplace=True)
us_city_location.head()

,city,lat,lng
0,"Prairie Ridge, WA",47.1443,-122.1408
1,"Edison, WA",48.5602,-122.4311
2,"Packwood, WA",46.6085,-121.6702
3,"Wautauga Beach, WA",47.5862,-122.5482
4,"Harper, WA",47.5207,-122.5196


In [11]:
## country format for food origin
food_origin = input("Food from: ")
print(food_origin)

Food from: Chile
Chile


In [12]:
country_location = country_centroids[['geounit', 'Latitude', 'Longitude']].copy()
country_location.head()

,geounit,Latitude,Longitude
0,Aruba,12.520880,-69.982677
1,Afghanistan,33.835231,66.004734
2,Angola,-12.293361,17.537368
3,Anguilla,18.223959,-63.064989
4,Albania,41.142450,20.049834


In [13]:
country_location.rename(columns = {'geounit': 'country', 'Latitude': 'lat', 'Longitude': 'lng'}, inplace=True)
country_location.head()

,country,lat,lng
0,Aruba,12.520880,-69.982677
1,Afghanistan,33.835231,66.004734
2,Angola,-12.293361,17.537368
3,Anguilla,18.223959,-63.064989
4,Albania,41.142450,20.049834


In [14]:
### example from geopy documentation: eventually want to snag lat/lng for city and food origin to calculate
newport_ri = (41.49008, -71.312796)
cleveland_oh = (41.499498, -81.695391)
dist_btw = great_circle(newport_ri, cleveland_oh).miles
print(dist_btw)

536.9979906964345


In [15]:
## to show user
print(f"Your food travelled approximately {dist_btw:.0f} miles")

Your food travelled approximately 537 miles


In [16]:
### values to multiply by great circle mile calculation

### ocean transport for refrigerated/temperature sensitive goods. Units = grams of CO2 per TEU kilometer (volume)
### reefer value multiplied by 0.621371 to get how many grams per mile
reefer = 941 * (0.621371)
### rail transport in grams of CO2 per TEU mile
rail = 292.8
### truck transport in grams of CO2 per TEU mile
truck = 597.4

In [17]:
### carbon emissions estimate for all transportation types
ghg_reefer = dist_btw * reefer
ghg_rail = dist_btw * rail
ghg_truck = dist_btw * truck

## {variable:.0f} for no decimal places
print(f"Carbon dioxide emissions are between: {ghg_rail:.0f}, {ghg_reefer:.0f}, and {ghg_truck:.0f} grams")

###TEU referencing a standard shipping container, meaning twenty-foot equivalent unit

Carbon dioxide emissions are between: 157233, 313988, and 320803 grams


In [18]:
### convert grams to pounds
### 1 gram = 0.00220462 lbs.
g_to_lbs = 0.00220462
ghg_rf_lbs = ghg_reefer * g_to_lbs
ghg_rl_lbs = ghg_rail * g_to_lbs
ghg_tk_lbs = ghg_truck * g_to_lbs

print(f"Carbon dioxide emissions are between: {ghg_rl_lbs:.0f}, {ghg_rf_lbs:.0f}, and {ghg_tk_lbs:.0f} pounds")

Carbon dioxide emissions are between: 347, 692, and 707 pounds


In [19]:
### water use
your_food = input("Enter your food here: ")
print(your_food)

Enter your food here: Apple
Apple


In [20]:
### look up food and proceed below
water_crops = pd.read_csv("C:/Users/Ashley/workspace/foodpact2/selected_crops_waterprint.csv")
water_crops.head()

,FAOSTAT crop code,product,green_global avg water footprint(m3ton-1),blue_global avg water footprint(m3ton-1),grey_global avg water footprint(m3ton-1),total_global avg water footprint(m3ton-1)
0,15,wheat,1277,342,207,1827
1,27,"rice, paddy",1146,341,187,1673
2,27,"rice, husked (brown)",1488,443,242,2172
3,44,barley,1213,79,131,1423
4,56,corn,947,81,194,1222


In [21]:
water_use = water_crops[['product', 'blue_global avg water footprint(m3ton-1)', 'total_global avg water footprint(m3ton-1)']].copy()
water_use.head()

,product,blue_global avg water footprint(m3ton-1),total_global avg water footprint(m3ton-1)
0,wheat,342,1827
1,"rice, paddy",341,1673
2,"rice, husked (brown)",443,2172
3,barley,79,1423
4,corn,81,1222


In [22]:
water_use.rename(columns = {'product': 'food', 'blue_global avg water footprint(m3ton-1)': 'blue water footprint',
                           'total_global avg water footprint(m3ton-1)': 'total water footprint'}, inplace=True)
water_use.head()
### units = metric tons of water

,food,blue water footprint,total water footprint
0,wheat,342,1827
1,"rice, paddy",341,1673
2,"rice, husked (brown)",443,2172
3,barley,79,1423
4,corn,81,1222


In [23]:
def ton_to_gal(x):
    return(x * 264.17)

water_use['blue gal'] = water_use['blue water footprint'].apply(ton_to_gal)
water_use.head()

,food,blue water footprint,total water footprint,blue gal
0,wheat,342,1827,90346.14
1,"rice, paddy",341,1673,90081.97
2,"rice, husked (brown)",443,2172,117027.31
3,barley,79,1423,20869.43
4,corn,81,1222,21397.77


In [24]:
water_use['total gal'] = water_use['total water footprint'].apply(ton_to_gal)
water_use.head()

,food,blue water footprint,total water footprint,blue gal,total gal
0,wheat,342,1827,90346.14,482638.59
1,"rice, paddy",341,1673,90081.97,441956.41
2,"rice, husked (brown)",443,2172,117027.31,573777.24
3,barley,79,1423,20869.43,375913.91
4,corn,81,1222,21397.77,322815.74


In [25]:
def ton_to_liters(x):
    return(x * 1000)
water_use['blue liters'] = water_use['blue water footprint'].apply(ton_to_liters)
water_use['total liters'] = water_use['total water footprint'].apply(ton_to_liters)
water_use.head()

,food,blue water footprint,total water footprint,blue gal,total gal,blue liters,total liters
0,wheat,342,1827,90346.14,482638.59,342000,1827000
1,"rice, paddy",341,1673,90081.97,441956.41,341000,1673000
2,"rice, husked (brown)",443,2172,117027.31,573777.24,443000,2172000
3,barley,79,1423,20869.43,375913.91,79000,1423000
4,corn,81,1222,21397.77,322815.74,81000,1222000


In [26]:
water_use.dtypes

food                      object
blue water footprint       int64
total water footprint      int64
blue gal                 float64
total gal                float64
blue liters                int64
total liters               int64
dtype: object

In [27]:
### This does NOT WORK:
### print(water_use[water_use['food']].str.contains('corn'))


### This WORKS, but returns boolean that says they are all 'True', which doesn't print out the associated rows with it
### print(water_use.food.str.contains("corn"))

In [28]:
def divider(x):
    return(x/2000)

In [29]:
water_use['blue tons per lb'] = water_use['blue water footprint'].apply(divider)

In [30]:
water_use['total tons per lb'] = water_use['total water footprint'].apply(divider)

In [31]:
water_use['blue gal per lb'] = water_use['blue gal'].apply(divider)

In [32]:
water_use['total gal per lb'] = water_use['total gal'].apply(divider)

In [33]:
water_use['blue liters per lb'] = water_use['blue liters'].apply(divider)
water_use['total liters per lb'] = water_use['total liters'].apply(divider)

In [34]:
water_use.head()

,food,blue water footprint,total water footprint,blue gal,total gal,blue liters,total liters,blue tons per lb,total tons per lb,blue gal per lb,total gal per lb,blue liters per lb,total liters per lb
0,wheat,342,1827,90346.14,482638.59,342000,1827000,0.1710,0.9135,45.173070,241.319295,171.0,913.5
1,"rice, paddy",341,1673,90081.97,441956.41,341000,1673000,0.1705,0.8365,45.040985,220.978205,170.5,836.5
2,"rice, husked (brown)",443,2172,117027.31,573777.24,443000,2172000,0.2215,1.0860,58.513655,286.888620,221.5,1086.0
3,barley,79,1423,20869.43,375913.91,79000,1423000,0.0395,0.7115,10.434715,187.956955,39.5,711.5
4,corn,81,1222,21397.77,322815.74,81000,1222000,0.0405,0.6110,10.698885,161.407870,40.5,611.0


In [35]:
### standard deviation for each numerical column
print(water_use.std())

blue water footprint     9.839598e+02
total water footprint    4.283762e+03
blue gal                 2.599327e+05
total gal                1.131641e+06
blue liters              9.839598e+05
total liters             4.283762e+06
blue tons per lb         4.919799e-01
total tons per lb        2.141881e+00
blue gal per lb          1.299663e+02
total gal per lb         5.658207e+02
blue liters per lb       4.919799e+02
total liters per lb      2.141881e+03
dtype: float64
